In [1]:
import tensorflow as tf
from swin_model import *
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
partition_layer = PatchPartition()
input_tensor = tf.random.uniform(shape=(1,224, 224, 3))
output_tensor = partition_layer(input_tensor)
output_tensor.shape # (1, (H/4 * W/4), (4 * 4 * 3)), for patch size = 4

TensorShape([1, 3136, 48])

In [3]:
embedding_layer = PatchEmbedding()
output_tensor = embedding_layer(output_tensor)
output_tensor.shape # (1, (H/4 * W/4), C)

TensorShape([1, 3136, 96])

In [4]:
merging_layer = PatchMerging((56, 56), 96)
output_tensor = merging_layer(output_tensor)
output_tensor.shape # (1, (H/8, W/8), 2C)

TensorShape([1, 784, 192])

In [5]:
wmsa = WindowAttention(192, (7, 7), 6)
new_tensor = tf.random.uniform((64, 49, 192))
new_output_tensor = wmsa(new_tensor)
new_output_tensor.shape

TensorShape([64, 49, 192])

In [6]:
tb = TransformerBlock(192, (28, 28), 6)
output_tensor = tb(output_tensor)
output_tensor.shape

TensorShape([1, 784, 192])